# Multi-Class Prediction of Cirrhosis Outcomes

### **Problem Statement**: 
#### The task is to predict survival state of patients with liver cirrhosis given features.
> Cirrhosis results from prolonged liver damage, leading to extensive scarring, often due to conditions like hepatitis or chronic alcohol consumption.

The original data description is adapted from this [notebook](https://www.kaggle.com/code/markuslill/s3e26-xgbclassifer) and the orignal data [source](https://www.kaggle.com/datasets/joebeachcapital/cirrhosis-patient-survival-prediction).

The data we are using currently differs from the original source.

| Variable Name | Role | Type | Encoding | Description | Missing Values |
| --- | --- | --- | --- | --- | --- |
| ID | ID | Int | drop | Unique identifier | No |
| N_Days | Feature? | Int | numeric | Number of days between registration and the earlier of death(D), transplantation(CL), or being alive at the study analysis time in July 1986(C) | No |
| Drug | Feature | Categorical | Binary | Type of drug: D-penicillamine or placebo. Type of medication may impact the effectiveness of treatment, thus affecting status. | Yes |
| Age | Feature | Int | numeric | Age. Age could be related to disease progression; older patients may have a different status trajectory. | No |
| Sex | Feature | Categorical | Binary | Gender: M (male) or F (female). Biological sex may influence disease patterns and response to treatment, thereby affecting status. | No |
| Ascites | Feature | Categorical | Binary | Presence of ascites: N (No) or Y (Yes). The accumulation of fluid in the abdomen, often a sign of advanced liver disease, which could indicate a poorer status. | Yes |
| Hepatomegaly | Feature | Categorical | Binary | Presence of hepatomegaly: N (No) or Y (Yes). Enlargement of the liver. If present, it might suggest more serious liver disease and potentially a poorer status. | Yes |
| Spiders | Feature | Categorical | Binary | Presence of spiders: N (No) or Y (Yes). Spider angiomas are small, spider-like capillaries visible under the skin, associated with liver disease and could indicate a more advanced disease affecting status. | Yes |
| Edema | Feature | Categorical | One-Hot | Presence of edema: N (no edema and no diuretic therapy for edema), S (edema present without diuretics, or edema resolved by diuretics), or Y (edema despite diuretic therapy). Swelling caused by excess fluid trapped in the body's tissues, often worsening the prognosis and indicating poorer status. | No |
| Bilirubin | Feature | Continuous(Int) | numeric | Serum bilirubin. High levels can indicate liver dysfunction and may correlate with more advanced disease and poorer status. | No |
| Cholesterol | Feature | Continous(Int)? | numeric | Serum cholesterol. While not directly related to liver function, abnormal levels can be associated with certain liver conditions and overall health status. | Yes |
| Albumin | Feature | Continous(Int) | numeric | Low levels can be a sign of liver disease and can indicate a poorer status due to the liver's reduced ability to synthesize proteins. | No |
| Copper | Feature | Int | numeric | Urine copper. Elevated in certain liver diseases (like Wilson's disease), and could affect status if levels are abnormally high. | Yes |
| Alk_Phos | Feature | Continuous | numeric | Alkaline phosphatase. An enzyme related to the bile ducts; high levels might indicate blockage or other issues related to the liver. | Yes |
| SGOT | Feature | Continuous | numeric | An enzyme that, when elevated, can indicate liver damage and could correlate with a worsening status. | Yes |
| Triglycerides | Feature | Int | numeric | Triglycerides. Though mainly a cardiovascular risk indicator, they can be affected by liver function and, by extension, the status of the patient. | Yes |
| Platelets | Feature | Int | numeric | Platelets per cubic. Low platelet count can be a result of advanced liver disease and can indicate a poorer status. | Yes |
| Prothrombin | Feature | Int | numeric | Prothrombin time. A measure of how long it takes blood to clot. Liver disease can cause increased times, indicating poorer status. | Yes |
| Stage | Feature | Categorical | Ordinal | UHistologic stage of disease (1, 2, 3, or 4). The stage of liver disease, which directly correlates with the patient's status - the higher the stage, the more serious the condition. | Yes |
| Status | Target | Categorical | Label | C (censored) indicates the patient was alive at N_Days, CL indicates the patient was alive at N_Days due to liver a transplant, and D indicates the patient was deceased at N_Days | No |

#### Glaring Issues in Dataset: 
1. Continous vs. Int
2. N_days is not considered as a feature but a 'other' type. It is indeed very confused, need some feature engineering to make better use of it.
3. Age is in days, no idea why, we could probably bin it.
4. Target class is imbalanced
5. feature engineering is needed
6. feature selection(RFE)
7. feature importance (SHAP) 

Feature Engineering ideas: <br>
|Transformer | Class | Type	| Description|
| --- | --- | --- | --- |
DiagnosisDateTransformer| num | Calculates 'Diagnosis_Date' by subtracting 'N_Days' from 'Age'. This could provide a more direct measure of time since diagnosis, relevant for analysis. |
AgeBinsTransformer|	cat	| Categorizes 'Age' into bins (19, 29, 49, 64, 99), converting a continuous variable into a categorical one for simplified analysis.|
BilirubinAlbuminTransformer| num | Creates a new feature 'Bilirubin_Albumin' by multiplying 'Bilirubin' and 'Albumin', potentially highlighting interactions between these two variables.|
NormalizeLabValuesTransformer|	num	 | Normalizes laboratory values (like 'Bilirubin', 'Cholesterol', etc.) to their z-scores, standardizing these features for modeling purposes.
DrugEffectivenessTransformer| num | Generates a new feature 'Drug_Effectiveness' by combining 'Drug' and 'Bilirubin' levels. This assumes that changes in 'Bilirubin' reflect drug effectiveness. |
SymptomScore(Cat)Transformer| num | Summarizes the presence of symptoms ('Ascites', 'Hepatomegaly', etc.) into a single 'Symptom_Score', simplifying the representation of patient symptoms. |
LiverFunctionTransformer| num | Computes 'Liver_Function_Index' as the average of key liver function tests, providing a comprehensive metric for liver health. |
RiskScoreTransformer| num | Calculates 'Risk_Score' using a combination of 'Bilirubin', 'Albumin', and 'Alk_Phos', potentially offering a composite risk assessment for patients.|
TimeFeaturesTransformer| num | Extracts 'Year' and 'Month' from 'N_Days', converting a continuous time measure into more interpretable categorical time units. |


## EDA

In [1]:
import warnings
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

alt.data_transformers.enable("vegafusion")

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
train_set = pd.read_csv("data/train.csv")
test_set = pd.read_csv("data/test.csv")

In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train_set, test_size=0.3, random_state=123)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5533 entries, 6987 to 3582
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5533 non-null   int64  
 1   N_Days         5533 non-null   int64  
 2   Drug           5533 non-null   object 
 3   Age            5533 non-null   int64  
 4   Sex            5533 non-null   object 
 5   Ascites        5533 non-null   object 
 6   Hepatomegaly   5533 non-null   object 
 7   Spiders        5533 non-null   object 
 8   Edema          5533 non-null   object 
 9   Bilirubin      5533 non-null   float64
 10  Cholesterol    5533 non-null   float64
 11  Albumin        5533 non-null   float64
 12  Copper         5533 non-null   float64
 13  Alk_Phos       5533 non-null   float64
 14  SGOT           5533 non-null   float64
 15  Tryglicerides  5533 non-null   float64
 16  Platelets      5533 non-null   float64
 17  Prothrombin    5533 non-null   float64
 18  Stage     

##### After a quick preliminary examination, we found no missing values from the provided train.csv. 

### Target

In [5]:
target = ['Status']

In [6]:
alt.Chart(train_df).mark_bar().encode(
    y = alt.Y('Status').sort('x'),
    x = 'count()'
).properties(
    title = "Histogram of Status, the target"
)

alt.Chart(...)

In [7]:
train_df['Status'].value_counts()

Status
C     3450
D     1888
CL     195
Name: count, dtype: int64

##### From the plot above, it's evident that there is a significant class imbalance, with only 3.48% of the data labeled as 'CL'. To address this issue, techniques such as over-sampling should be considered to ensure a more balanced dataset.

##### Let's examine the distribution of the specified features in relation to the target variable to identify any notable patterns.

### Categorical Features 

In [8]:
cat_col = ["Drug", "Sex", "Ascites", "Hepatomegaly", "Edema", "Spiders", "Stage"]

In [9]:
chart = alt.Chart(train_df).mark_bar().encode(
    alt.X(alt.repeat()),
    y = 'count()',
    color = 'Status'
).properties(
    width = 100,
    height = 200
).repeat(
        cat_col
)

chart.properties(
    title="Histograms of Categorical Features relative to Status"
)

alt.RepeatChart(...)

### Numerical Features 

In [10]:
numeric_col = ["N_Days", "Age", "Bilirubin", "Albumin", "Copper", "Alk_Phos", "SGOT", "Tryglicerides", "Platelets", "Prothrombin"]

In [11]:
chart = alt.Chart(train_df).mark_boxplot().encode(
    x = alt.X(alt.repeat()).type('quantitative'),
    color = 'Status'
).properties(
    width = 300,
    height = 200
).repeat(
        numeric_col
)

chart.properties(
    title="Boxplots of Numerical Features relative to Status"
)

alt.RepeatChart(...)

From what we've seen, there's no clear link between the target and the other features. <br>
Among numerical features, `N_Days` is the number of days between the registeration date and the date of study. This feature is confusing and could be misleading if we interpret it, so we decide to remove it. <br>

Also, we observed that the unit of `Age` is in days, not in years. We could bin the data to different categories:

    - Infants and Toddlers: 0 to 730 days (0 - 2 years)
    - Children: 1,095 to 3,650 days (3 - 10 years)
    - Adolescents: 4,015 to 6,205 days (11 - 17 years)
    - Young Adults: 6,570 to 10,585 days (18 - 29 years)
    - Adults: 10,950 to 14,235 days (30 - 39 years)
    - Middle-Aged Adults: 14,600 to 17,885 days (40 - 49 years)
    - Pre-Senior Adults: 18,250 to 21,535 days (50 - 59 years)
    - Senior Adults: 21,900 to 25,185 days (60 - 69 years)
    - Elderly: 25,550 to 28,835 days (70 - 79 years)
    - Advanced Elderly: 29,200 days and above (80 years and above)
    
<mark> What's more, we've spotted some outliers in the numerical data. It might be a good idea to think about taking them out for a cleaner analysis.???? borrowed from somewhere else, I don't think we should remove the outliers, we can certainly try, but I will not include it into our processing for now </mark>

### Correlation Matrix

##### Let's examine the correlation matrix of the numerical features to obtain a clearer insight into their interrelationships.

In [12]:
correlation_matrix_numeric = (
    train_df[numeric_col + target]
    .corr(numeric_only=True)
    .style.background_gradient(cmap="viridis")
    .set_table_styles(
        [
            {
                "selector": "table",
                "props": [
                    ("max-width", "50px"),
                    ("max-height", "50px"),
                    ("font-size", "2px"),
                ],
            }
        ]
    )
)
correlation_matrix_numeric

,N_Days,Age,Bilirubin,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
N_Days,1.000000,-0.096595,-0.352691,0.254242,-0.293150,-0.034437,-0.229880,-0.178538,0.147968,-0.169240
Age,-0.096595,1.000000,0.092965,-0.126071,0.091612,0.017462,-0.027498,0.005433,-0.095516,0.140256
Bilirubin,-0.352691,0.092965,1.000000,-0.307063,0.454320,0.133211,0.368615,0.331292,-0.072335,0.304078
Albumin,0.254242,-0.126071,-0.307063,1.000000,-0.224307,-0.096815,-0.202210,-0.110924,0.139159,-0.194037
Copper,-0.293150,0.091612,0.454320,-0.224307,1.000000,0.125929,0.318272,0.300082,-0.096298,0.255287
Alk_Phos,-0.034437,0.017462,0.133211,-0.096815,0.125929,1.000000,0.130860,0.095902,0.038205,0.091082
SGOT,-0.229880,-0.027498,0.368615,-0.202210,0.318272,0.130860,1.000000,0.164774,-0.048273,0.138497
Tryglicerides,-0.178538,0.005433,0.331292,-0.110924,0.300082,0.095902,0.164774,1.000000,0.013973,0.074296
Platelets,0.147968,-0.095516,-0.072335,0.139159,-0.096298,0.038205,-0.048273,0.013973,1.000000,-0.159679
Prothrombin,-0.169240,0.140256,0.304078,-0.194037,0.255287,0.091082,0.138497,0.074296,-0.159679,1.000000


##### From the correlation matrix, we can observe that `Bilirubin` and `Copper` exhibit the strongest correlation, with a score of 0.442223. Additionally, `Bilirubin`, `Albumin`, and `SGOT` demonstrate moderate correlations, each surpassing the 0.3. While these features don't display strong correlations, further analysis is needed to confirm that they are not affected by multicollinearity.

### Summary of EDA

- Among numerical columns, `ID` and `N_Days` carry minimal useful information, which could hinder the performance of our model. We decide to drop them for our analysis.
- The `Age` column should be binned into different age groups for better intrepretation of the data.
- We saw no obvious high correlation between any feature with the target and between features themselves. Among all the correlations, `Bilirubin` and `Copper`, `Bilirubin`, `Albumin`, and `SGOT` stood out, which can be furtherly analyzed with enough time. 

As required, our model is measure based on the **log-loss** metric.

## Feature Engineering

In [13]:
#encoding
#remove outlier
#new feature
#pipe+resample

##### In this section, we'll start by introducing new features that are relevant to our specific task, aiming to enhance our analytical capabilities. Following that, we will proceed with encoding all the features and performing the necessary preprocessing steps.

**Additional Features** we aim to extract: 
- **AGE (Discretizing the Age Column)** The logic window as below will be used:
    - Infants and Toddlers: 0 to 730 days (0 - 2 years)
    - Children: 1,095 to 3,650 days (3 - 10 years)
    - Adolescents: 4,015 to 6,205 days (11 - 17 years)
    - Young Adults: 6,570 to 10,585 days (18 - 29 years)
    - Adults: 10,950 to 14,235 days (30 - 39 years)
    - Middle-Aged Adults: 14,600 to 17,885 days (40 - 49 years)
    - Pre-Senior Adults: 18,250 to 21,535 days (50 - 59 years)
    - Senior Adults: 21,900 to 25,185 days (60 - 69 years)
    - Elderly: 25,550 to 28,835 days (70 - 79 years)
    - Advanced Elderly: 29,200 days and above (80 years and above)
 
Borrowing from this [notebook](https://www.kaggle.com/code/markuslill/s3e26-xgbclassifer), we decide to incoparate the following features:
- **DrugEffectiveness** This feature combines `Drug` and `Bilirubin` levels. This assumes that changes in 'Bilirubin' reflect drug effectiveness. We add an interaction term of these two features: `Drug` * `Bilirubin`. 
- **SymptomScore** This feature summarizes the presence of symptoms ['Ascites', 'Hepatomegaly', 'Spiders', 'Edema_S', 'Edema_Y'] into a single 'Symptom_Score', simplifying the representation of patient symptoms. <mark>('Edema_N') should be removed from the list cuz i think its not a bad symptom. </mark>
- **LiverFunction**	This feature is average of key liver function test: ['Bilirubin', 'Albumin', 'Alk_Phos', 'SGOT']:(X['Liver_Function_Index'] = X[liver_columns].mean(axis=1)), providing a comprehensive metric for liver health.	
- **RiskScore** This feature is 'Risk_Score' using a combination of 'Bilirubin', 'Albumin', and 'Alk_Phos', (X['Bilirubin'] + X['Albumin'] - X['Alk_Phos']), potentially offering a composite risk assessment for patients.

**Note**: We'll start by keeping all the original columns. If we encouter the curse of dimensionality, we'll tackle it by identifying and removing less important features later in the process.

### Encoding

In [14]:
numeric_feat = ["Age", "Bilirubin", "Albumin", "Copper", "Alk_Phos", "SGOT", "Tryglicerides", "Platelets", "Prothrombin"]
bin_feat = ["Age"]
onehot_feat = ["Edema"]
ordinal_feat = ["Stage"]
binary_feat = ["Drug", "Sex", "Ascites", "Hepatomegaly", "Spiders"]
drop_feat = ["N_Days", "id"]
target = ["Status"]

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin
class BilirubinAlbuminTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        bilirubin_index = 1  # Replace with actual index of 'Bilirubin'
        albumin_index = 2   # Replace with actual index of 'Albumin'
        bilirubin_albumin = X[:, bilirubin_index] * X[:, albumin_index]
        return np.column_stack((X, bilirubin_albumin))

class DrugEffectivenessTransformer(BaseEstimator, TransformerMixin):
    # Placeholder concept, assuming 'Bilirubin' improvement is a measure of effectiveness
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        bilirubin_index = 1  # Replace with actual index of 'Bilirubin'
        drug_index = 23   # Replace with actual index of 'Albumin'
        
        drug_bilirubin = X[:, bilirubin_index] * X[:, drug_index]
        return np.column_stack((X, drug_bilirubin))
class SymptomScoreTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Assuming the indices of the columns are known
        symptom_indices = [25, 26, 27, 20, 21]  # Replace with actual indices
        X_symptom_score = X[:, symptom_indices].sum(axis=1).reshape(-1, 1)
        return np.hstack((X, X_symptom_score))

class LiverFunctionTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        liver_indices = [1, 2, 4, 5]  # Replace with actual indices
        X_liver_function_index = X[:, liver_indices].mean(axis=1).reshape(-1, 1)
        return np.hstack((X, X_liver_function_index))

class RiskScoreTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Indices for Bilirubin, Albumin, Alk_Phos
        bilirubin_index, albumin_index, alk_phos_index = 1, 2, 4  # Replace with actual indices
        X_risk_score = X[:, bilirubin_index] + X[:, albumin_index] - X[:, alk_phos_index]
        X_risk_score = X_risk_score.reshape(-1, 1)
        return np.hstack((X, X_risk_score))

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline

# Set up the initial column transformer
initial_preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_feat),
        ('binner', KBinsDiscretizer(n_bins=10, encode="onehot"), bin_feat),
        ('onehot', OneHotEncoder(handle_unknown="ignore", sparse=False), onehot_feat),
        ('ordinal', OrdinalEncoder(dtype=int), ordinal_feat),
        ('binary', OneHotEncoder(handle_unknown="ignore", sparse=False, drop='if_binary'), binary_feat),
        ('drop_columns', 'drop', drop_feat)
    ]
)

# Create a pipeline that includes the initial preprocessing and custom transformations
final_preprocessor = Pipeline(steps=[
    ('initial_preprocessing', initial_preprocessor),
    ('bilirubin_albumin', BilirubinAlbuminTransformer()),
    ('drug_effectiveness', DrugEffectivenessTransformer()),
    ('symptom_score', SymptomScoreTransformer()),
    ('liver_function', LiverFunctionTransformer()),
    ('risk_score', RiskScoreTransformer())
])

In [17]:
X_train = train_df.drop(columns=["Status"])
y_train = train_df["Status"]
X_test = test_df.drop(columns=["Status"])
y_test = test_df["Status"]
X_train.shape

(5533, 19)

In [18]:
X_train

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
6987,6987,2576,D-penicillamine,19722,F,N,N,N,N,0.5,233.0,3.54,20.0,663.0,82.15,62.0,164.0,11.0,1.0
5779,5779,3820,Placebo,11872,F,N,Y,N,N,1.8,394.0,3.08,75.0,2132.0,102.30,103.0,388.0,10.0,4.0
6105,6105,3584,D-penicillamine,15574,F,N,N,N,N,0.7,266.0,3.61,30.0,1584.0,111.60,85.0,307.0,10.3,3.0
7578,7578,1945,D-penicillamine,14106,F,N,N,Y,N,1.0,309.0,3.66,77.0,1214.0,158.10,101.0,309.0,9.7,3.0
2611,2611,2443,Placebo,16688,F,N,Y,N,N,0.5,267.0,3.60,41.0,1142.0,70.00,130.0,336.0,9.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4060,4060,1321,Placebo,18302,F,N,N,N,N,0.8,257.0,3.89,9.0,637.0,97.65,146.0,295.0,10.6,1.0
1346,1346,3422,D-penicillamine,23107,F,N,Y,Y,N,14.5,448.0,3.06,34.0,1218.0,60.45,318.0,385.0,11.4,4.0
3454,3454,1615,Placebo,19155,F,N,Y,Y,N,0.7,303.0,3.35,42.0,2132.0,79.05,91.0,251.0,11.7,4.0
7533,7533,2468,Placebo,17233,F,N,N,N,N,1.4,201.0,3.94,44.0,1345.0,54.25,145.0,445.0,11.2,3.0


In [19]:
transformed_data = final_preprocessor.fit_transform(X_train)
transformed_test = final_preprocessor.transform(X_test)

In [20]:
# Extracting feature names for KBinsDiscretizer
bin_feature_names = initial_preprocessor.named_transformers_["binner"].get_feature_names_out(input_features=bin_feat)

# Extracting feature names for OneHotEncoder
onehot_feature_names = initial_preprocessor.named_transformers_["onehot"].get_feature_names_out(input_features=onehot_feat)

# Extracting feature names for ordinal features
ordinal_feature_names = initial_preprocessor.named_transformers_["ordinal"].get_feature_names_out(input_features=ordinal_feat)

# Extracting feature names for binary features
binary_feature_names = initial_preprocessor.named_transformers_["binary"].get_feature_names_out(input_features=binary_feat)

# Combine all feature names
all_feature_names = numeric_feat + bin_feature_names.tolist() + onehot_feature_names.tolist() + ordinal_feature_names.tolist()+ binary_feature_names.tolist()

# Creating a DataFrame with the transformed data and feature names
custom_feature_names = ['Bilirubin_Albumin', 'DrugEffectiveness', 'Symptom_Score', 'Liver_Function_Index', 'Risk_Score']

# Now combine all feature names including those from custom transformers
all_feature_names.extend(custom_feature_names)

# Creating a DataFrame with the transformed data and all feature names
transformed_df = pd.DataFrame(transformed_data, columns=all_feature_names)
transformed_df.columns

Index(['Age', 'Bilirubin', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT',
       'Tryglicerides', 'Platelets', 'Prothrombin', 'Age_0.0', 'Age_1.0',
       'Age_2.0', 'Age_3.0', 'Age_4.0', 'Age_5.0', 'Age_6.0', 'Age_7.0',
       'Age_8.0', 'Age_9.0', 'Edema_N', 'Edema_S', 'Edema_Y', 'Stage',
       'Drug_Placebo', 'Sex_M', 'Ascites_Y', 'Hepatomegaly_Y', 'Spiders_Y',
       'Bilirubin_Albumin', 'DrugEffectiveness', 'Symptom_Score',
       'Liver_Function_Index', 'Risk_Score'],
      dtype='object')

In [21]:
transformed_df

,Age,Bilirubin,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Age_0.0,...,Drug_Placebo,Sex_M,Ascites_Y,Hepatomegaly_Y,Spiders_Y,Bilirubin_Albumin,DrugEffectiveness,Symptom_Score,Liver_Function_Index,Risk_Score
0,0.369760,-0.549650,-0.018708,-0.844981,-0.610742,-0.668748,-0.996171,-1.158536,0.467200,0.0,...,0.0,0.0,0.0,0.0,0.0,0.010283,-0.000000,0.0,-0.461962,0.042385
1,-1.756912,-0.215275,-1.346512,-0.121868,0.155319,-0.256747,-0.232723,1.405645,-0.806207,1.0,...,1.0,0.0,0.0,1.0,0.0,0.289871,-0.215275,1.0,-0.415804,-1.717106
2,-0.753990,-0.498207,0.183349,-0.713506,-0.130455,-0.066593,-0.567895,0.478419,-0.424185,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.091346,-0.000000,0.0,-0.127977,-0.184403
3,-1.151691,-0.421044,0.327676,-0.095573,-0.323404,0.884176,-0.269965,0.501314,-1.188229,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.137966,-0.000000,1.0,0.116851,0.230036
4,-0.452192,-0.549650,0.154484,-0.568883,-0.360951,-0.917174,0.270035,0.810389,-0.933548,0.0,...,1.0,0.0,0.0,1.0,0.0,-0.084912,-0.549650,1.0,-0.418323,-0.034214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,-0.014938,-0.472486,0.991578,-0.989603,-0.624301,-0.351824,0.567966,0.341052,-0.042163,0.0,...,1.0,0.0,0.0,0.0,0.0,-0.468507,-0.472486,0.0,-0.114258,1.143393
5529,1.286802,3.051304,-1.404243,-0.660916,-0.321318,-1.112440,3.770723,1.371304,0.976563,0.0,...,0.0,0.0,0.0,1.0,1.0,-4.284771,0.000000,2.0,0.053326,1.968379
5530,0.216152,-0.498207,-0.567149,-0.555736,0.155319,-0.732132,-0.456171,-0.162626,1.358585,0.0,...,1.0,0.0,0.0,1.0,1.0,0.282558,-0.498207,2.0,-0.410542,-1.220675
5531,-0.304544,-0.318160,1.135904,-0.529441,-0.255090,-1.239209,0.549345,2.058138,0.721881,0.0,...,1.0,0.0,0.0,0.0,0.0,-0.361399,-0.318160,0.0,-0.169139,1.072835


In [22]:
y_train

6987    C
5779    C
6105    C
7578    C
2611    C
       ..
4060    C
1346    D
3454    C
7533    C
3582    C
Name: Status, Length: 5533, dtype: object

In [23]:
y_train_transformed = y_train.map({'C': 0, 'CL': 1, 'D': 2})
y_train_transformed

6987    0
5779    0
6105    0
7578    0
2611    0
       ..
4060    0
1346    2
3454    0
7533    0
3582    0
Name: Status, Length: 5533, dtype: int64

## Baseline Model

#### TO-DO
##### remove outliers (✓ data size is too small)
##### log loss function for hyperparameter tuning (✓)
##### feature importance after transformation
##### feature selection after model (rse, embedded method)
##### kernelized SVM (✓), xgboost, random forest
##### stratified CV (✓ but the result gets worse)

### Logistic regression

In [24]:
#!pip install imblearn

In [25]:
X_train = train_df.drop(columns=["Status"])
y_train = train_df["Status"]
X_test = test_df.drop(columns=["Status"])
y_test = test_df["Status"]
X_train_transformed = X_train
X_train_transformed.shape

(5533, 19)

In [26]:
# X_train

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbpipeline

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_transformed, test_size=0.3, random_state=123)
# X_train.shape, X_val.shape, y_train.shape, y_val.shape
model_lr = LogisticRegression(multi_class="multinomial", random_state = 123, class_weight = "balanced", max_iter = 3000)
pipe_lr = imbpipeline([
    ('initial_preprocessing', initial_preprocessor),
    ('bilirubin_albumin', BilirubinAlbuminTransformer()),
    #('drug_effectiveness', DrugEffectivenessTransformer()),
    ('symptom_score', SymptomScoreTransformer()),
    ('liver_function', LiverFunctionTransformer()),
    ('risk_score', RiskScoreTransformer()),
    ('smote', SMOTE(random_state=123)),
    ('logistic_regression', model_lr)
])

In [28]:
from sklearn.model_selection import cross_validate
cross_val_results = {}
cross_val_results["logreg"] = (
    pd.DataFrame(
        cross_validate(
            pipe_lr,
            X_train,
            y_train,
            scoring="neg_log_loss",
            return_train_score=True,
            # cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
        )
    )
    .agg(["mean", "std"])
    .round(3)
    .T
)

In [29]:
cross_val_results["logreg"]

,mean,std
fit_time,0.525,0.023
score_time,0.003,0.000
test_score,-0.802,0.034
train_score,-0.779,0.009


In [30]:
param_grid = {
    "logistic_regression__C":[0.01, 0.1, 1, 10, 100],
    "logistic_regression__solver":["newton-cg", "lbfgs", "saga", "sag"]
}

grid_search = GridSearchCV(pipe_lr, param_grid, cv=5, scoring="neg_log_loss")
grid_search.fit(X_train, y_train)

print("Best hyperparameters are ", grid_search.best_params_)

prediction = grid_search.predict(X_val)
probability = grid_search.predict_proba(X_val)
print("Log Loss is ", log_loss(y_val, probability))
print(classification_report(y_val, prediction))

Best hyperparameters are  {'logistic_regression__C': 0.01, 'logistic_regression__solver': 'lbfgs'}
Log Loss is  0.7735651736490685
              precision    recall  f1-score   support

           0       0.88      0.72      0.80      1024
           1       0.11      0.61      0.19        64
           2       0.79      0.66      0.72       572

    accuracy                           0.70      1660
   macro avg       0.60      0.66      0.57      1660
weighted avg       0.82      0.70      0.75      1660



### Kernelized SVM

In [31]:
from sklearn.svm import SVC

model_svm = SVC(probability=True)

pipe_svm = imbpipeline([
    ('initial_preprocessing', initial_preprocessor),
    ('bilirubin_albumin', BilirubinAlbuminTransformer()),
    #('drug_effectiveness', DrugEffectivenessTransformer()),
    ('symptom_score', SymptomScoreTransformer()),
    ('liver_function', LiverFunctionTransformer()),
    ('risk_score', RiskScoreTransformer()),
    ('smote', SMOTE(random_state=123)),
    ('svm', model_svm)
])

param_grid = {
    "svm__C":[0.1, 1, 10],
    "svm__kernel":["rbf", "poly", "sigmoid"],
    "svm__gamma":["scale", "auto"]
}

grid_search = GridSearchCV(pipe_svm, param_grid, cv=5, scoring="neg_log_loss")
grid_search.fit(X_train, y_train)

print("Best hyperparameters are ", grid_search.best_params_)

prediction = grid_search.predict(X_val)
probability = grid_search.predict_proba(X_val)
print("Log Loss is ", log_loss(y_val, probability))
print(classification_report(y_val, prediction))

KeyboardInterrupt: 

### Different models overviews

In [32]:
from sklearn.ensemble import AdaBoostRegressor, VotingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, RandomForestClassifier
import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier, log_evaluation, early_stopping
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

def fit_and_evaluate(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    model_pred = model.predict_proba(X_val)
    model_loss = log_loss(y_val, model_pred)
    return model_loss

# Define models
models = {
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=10),
    'Gradient Boosted': GradientBoostingClassifier(random_state=60),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=60), 
    'AdaBoost': AdaBoostClassifier(random_state=60),
    "logistic_regression": LogisticRegression(multi_class="multinomial", random_state = 123, class_weight = "balanced", max_iter = 3000),
    "random_forest": RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2,
                                             min_samples_leaf=1, max_features='sqrt', bootstrap=True, random_state=123),
    "lgbm": LGBMClassifier(n_estimators=100, learning_rate=0.1, num_leaves=31, random_state=123, verbose=-1),
    "xgb": XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=123, verbosity=0),
    "catboost": CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, random_state=123, verbose=0),
    "svm": SVC(C=1.0, kernel='rbf', gamma='scale', probability=True)
}
# Train and evaluate each model
#X_train, X_val, y_train, y_val = train_test_split(transformed_df, y_train_transformed, test_size=0.3, random_state=123)
loss_scores = {}
for name, model in models.items():
    pipe = imbpipeline([
        ('initial_preprocessing', initial_preprocessor),
        ('bilirubin_albumin', BilirubinAlbuminTransformer()),
        ('drug_effectiveness', DrugEffectivenessTransformer()),
        ('symptom_score', SymptomScoreTransformer()),
        ('liver_function', LiverFunctionTransformer()),
        ('risk_score', RiskScoreTransformer()),
        ('smote', SMOTE(random_state=123)),
        (name, model)
    ])
    score = fit_and_evaluate(pipe, X_train, y_train, X_val, y_val)
    loss_scores[name] = score
    
    
# check the result
loss_scores

{'K-Nearest Neighbors': 3.3292163499313383,
 'Gradient Boosted': 0.5038725127511171,
 'Naive Bayes': 6.452879471351502,
 'Decision Tree': 10.340826426669043,
 'AdaBoost': 1.0495590453765677,
 'logistic_regression': 0.7835504403980712,
 'random_forest': 0.5460254106070523,
 'lgbm': 0.4991402856534089,
 'xgb': 0.530244204008472,
 'catboost': 0.569273064604815,
 'svm': 0.6285647261100645}

In [46]:
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import Ridge
models_Voting_Stacking = ({
    "voting": VotingClassifier(estimators=list(models.items()), voting = 'soft'),
    "stacking": StackingClassifier(estimators=list(models.items()), final_estimator=LogisticRegression())
})

In [47]:
for name, model in models_Voting_Stacking.items():
    pipe = imbpipeline([
        ('initial_preprocessing', initial_preprocessor),
        ('bilirubin_albumin', BilirubinAlbuminTransformer()),
        ('drug_effectiveness', DrugEffectivenessTransformer()),
        ('symptom_score', SymptomScoreTransformer()),
        ('liver_function', LiverFunctionTransformer()),
        ('risk_score', RiskScoreTransformer()),
        ('smote', SMOTE(random_state=123)),
        (name, model)
    ])
    score = fit_and_evaluate(pipe, X_train, y_train, X_val, y_val)
    loss_scores[name] = score
loss_scores

/Users/yimeiyang/miniforge3/envs/573/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'K-Nearest Neighbors': 3.3292163499313383,
 'Gradient Boosted': 0.5038725127511171,
 'Naive Bayes': 6.452879471351502,
 'Decision Tree': 10.340826426669043,
 'AdaBoost': 1.0495590453765677,
 'logistic_regression': 0.7835504403980712,
 'random_forest': 0.5460254106070523,
 'lgbm': 0.4991402856534089,
 'xgb': 0.530244204008472,
 'catboost': 0.569273064604815,
 'svm': 0.6285647261100645,
 'voting': 0.6068019031189449,
 'stacking': 0.61399266453812}

In [34]:
#best model before tuning: 
best_mode_pre_tune = LGBMClassifier(n_estimators=100, learning_rate=0.1, num_leaves=31, random_state=123, verbose=-1)

## Hyperparameter Tuning

In [49]:
param_grid = {
    "lgbm": {
        "lgbm__n_estimators": [50, 100, 150, 200, 300],
        "lgbm__learning_rate": [0.01, 0.05, 0.1],
        "lgbm__num_leaves": [31, 50, 100]
    }
}


In [51]:
best_model ={"lgbm": LGBMClassifier(n_estimators=100, learning_rate=0.1, num_leaves=31, random_state=123, verbose=-1)}

In [ ]:
results = {}
for name, model in best_model.items():
    print(f"Processing and training model: {name}")
    pipe = imbpipeline([
        ('initial_preprocessing', initial_preprocessor),
        ('bilirubin_albumin', BilirubinAlbuminTransformer()),
        ('drug_effectiveness', DrugEffectivenessTransformer()),
        ('symptom_score', SymptomScoreTransformer()),
        ('liver_function', LiverFunctionTransformer()),
        ('risk_score', RiskScoreTransformer()),
        ('smote', SMOTE(random_state=123)),
        (name, model)
    ])
    grid_search = GridSearchCV(pipe, param_grid[name], cv=5, scoring="neg_log_loss", return_train_score=True)
    grid_search.fit(X_train, y_train)
    results[name] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'cv_results': grid_search.cv_results_
    }

Processing and training model: lgbm


In [ ]:
pd.DataFrame(results).T

### Test data prediction

In [36]:
# test
test_data = test_set.drop(['id'], axis=1)

In [37]:
test_data.shape

(5271, 18)

In [ ]:
# Predictions with the model
test_predictions = model.predict_proba(test_data)

# DataFrame for submission
submission_df = pd.DataFrame({
    'id': test['id'],
    'Status_C': test_predictions[:, 0],
    'Status_CL': test_predictions[:, 1],
    'Status_D': test_predictions[:, 2]
})

submission_df.to_csv('submission.csv', index=False)